# 숫자 필드
- 숫자 값을 저장하는 스칼라 필드
    - BOOL: Ture, False
    - INT8: 8비트 정수, 작은 범위의 정수 데이터를 저장
    - INT16: 16비트 정수, 중간 범위 정수 데이터 적합
    - INT32: 32비트 정수, 제품 수량이나 사용자 ID와 같은 일반적인 정수 데이터 저장에 적합
    - INT64: 64비트 정수, 타임스탬프나 식별자와 같은 대용량 데이터 저장에 적합
    - FLOAT: 32비트 부동소수점 숫자, 등급이나 온도와 같이 일반적인 정밀도가 필요한 데이터에 적합
    - DOUBLE: 64비트 배정밀도 부동 소수점 숫자, 재무 정보나 과학계산과 같은 고정밀 데이터에 적합

- 참고 자료: https://milvus.io/docs/ko/number.md

## 숫자 필드 추가
- 숫자 데이터를 저장하려면 컬렉션 스키마에 숫자 필드를 정의
    - age: 정수 데이터 저장, null 허용, 기본값 18
    - price: 부동 소수점 데이터 저장, null 허용 기본값 x

In [7]:
from pymilvus import MilvusClient, DataType

SERVER_ADDR = "http://localhost:19530"

client = MilvusClient(uri=SERVER_ADDR)

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

schema.add_field(field_name="age", datatype=DataType.INT64, nullable=True, default_value=18)
schema.add_field(field_name="price", datatype=DataType.FLOAT, nullable=True)
schema.add_field(field_name="pk", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=3)

{'auto_id': False, 'description': '', 'fields': [{'name': 'age', 'description': '', 'type': <DataType.INT64: 5>, 'default_value': long_data: 18
, 'nullable': True}, {'name': 'price', 'description': '', 'type': <DataType.FLOAT: 10>, 'nullable': True}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 3}}], 'enable_dynamic_field': True}

## 인덱스 매개변수 설정

In [8]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="age",
    index_type="AUTOINDEX",
    index_name="age_index"
)

index_params.add_index(
    field_name="embedding",
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

## 컬렉션 만들기

In [9]:
if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params
)

## 데이터 삽입

In [10]:
# Sample data
data = [
    {"age": 25, "price": 99.99, "pk": 1, "embedding": [0.1, 0.2, 0.3]},
    {"age": 30, "pk": 2, "embedding": [0.4, 0.5, 0.6]}, # `price` field is missing, which should be null
    {"age": None, "price": None, "pk": 3, "embedding": [0.2, 0.3, 0.1]},  # `age` should default to 18, `price` is null
    {"age": 45, "price": None, "pk": 4, "embedding": [0.9, 0.1, 0.4]},  # `price` is null
    {"age": None, "price": 59.99, "pk": 5, "embedding": [0.8, 0.5, 0.3]},  # `age` should default to 18
    {"age": 60, "price": None, "pk": 6, "embedding": [0.1, 0.6, 0.9]}  # `price` is null
]

client.insert(
    collection_name="my_collection",
    data=data
)


{'insert_count': 6, 'ids': [1, 2, 3, 4, 5, 6], 'cost': 0}

# 필터 표현식을 사용하여 쿼리

In [12]:
filter = "age > 30"

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["age", "price", "pk"]
)

print(res)

data: ["{'age': 45, 'price': None, 'pk': 4}", "{'age': 60, 'price': None, 'pk': 6}"]


In [16]:
filter = 'price is null'

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["age", "price", "pk"]
)

print(res)

data: ["{'age': 30, 'price': None, 'pk': 2}", "{'age': 18, 'price': None, 'pk': 3}", "{'age': 45, 'price': None, 'pk': 4}", "{'age': 60, 'price': None, 'pk': 6}"]


In [19]:
filter = 'price is null and age < 20'

res = client.query(
    collection_name="my_collection",
    filter=filter,
    output_fields=["age", "price", "pk"]
)

print(res)

data: ["{'pk': 3, 'age': 18, 'price': None}"]


# 필터 표현식을 사용한 벡터 검색

In [21]:
filter = "25 <= age <= 35"

res = client.search(
    collection_name="my_collection",
    data=[[0.3, -0.6, 0.1]],
    limit=5,
    search_params={"params": {"nprobe": 10}},
    output_fields=["age", "price"],
    filter=filter
)

print(res)

data: [[{'pk': 2, 'distance': -0.2016308456659317, 'entity': {'age': 30, 'price': None}}, {'pk': 1, 'distance': -0.23643313348293304, 'entity': {'age': 25, 'price': 99.98999786376953}}]]
